<a href="https://colab.research.google.com/github/IntersysConsulting/apex-api-impl-demo/blob/master/Capstone_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Segmenting and Clustering Neighborhoods in Toronto

##Requirements
Start by creating a new Notebook for this assignment. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

### Import libraries

In [0]:
# importing necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Scrap the webpage and read the table

In [36]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
lst = pd.read_html(str(table))
df = lst[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Clean df removing rows with Not assigned value in Borough

In [0]:
df = df[df.Borough != 'Not assigned']

### If Not assigned value remains in Neighbourhood lets replace it with the value in Borough

In [38]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Group and concatenate all the neighbourhoods with same Borough and Postcode

In [39]:
df['Neighbourhood'] = df.groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
df = df.drop_duplicates()
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park


In [40]:
df.shape


(103, 3)

Try to get geo values

In [0]:
df.reset_index(drop=True,inplace=True)

In [42]:
from geopy.geocoders import Nominatim
import folium
geolocator = Nominatim(timeout=10, user_agent = "dlab.berkeley.edu-workshop")
location = geolocator.geocode('M6A')
location

Location(M6A, Market Street, Little Germany, Bradford, Yorkshire and the Humber, England, BD1 1LH, United Kingdom, (53.7941642, -1.7520058, 0.0))

In [43]:
geocodes = [geolocator.geocode(', '.join([df['Neighbourhood'][i],'Toronto']))for i in range(len(df))]
notfound = list(filter(lambda g: g == None, geocodes))
len(notfound)

43

After several failed tries, well get the values from given csv

In [44]:
geodata = pd.read_csv('https://cocl.us/Geospatial_data')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [0]:
df.sort_values(by=['Postcode'],inplace=True)

In [0]:
df['Latitude'] = geodata['Latitude']
df['Longitude'] = geodata['Longitude']

In [48]:
df.reset_index(drop=True,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.727929,-79.262029
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
